## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Execution import execute_circuits

## Experiments

In [ ]:
qec_cycle_count_list = [1,2,4,8,16,32,64]
lqc_list = []

In [ ]:
lqc_base = LogicalCircuit(1, **steane_code)
lqc_base.encode([0], max_iterations=1)

### Without measurement error correction

In [ ]:
lqc_list = []

for n_qec_cycles in qec_cycle_count_list:
    lqc = lqc_base.copy()
    for _ in range(n_qec_cycles):
        lqc.append_qec_cycle()
    lqc.measure_all(with_error_correction=False)
    lqc_list.append(lqc)

lqc_list[0].draw("mpl")

In [ ]:
shots = 1E5
results = execute_circuits(
  lqc_list,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None,
  method="statevector", shots=shots
)

In [ ]:
# Option 1: Logical Counts
logical_counts_list = [lqc.get_logical_counts(result.get_counts()) for lqc, result in zip(lqc_list, results)]
infidelities_noqec = [logical_counts.get("1", 0)/shots for logical_counts in logical_counts_list]

# Option 2: LogicalStatevector
# infidelities_noqec = []
# for result in results:
#     lsv = LogicalStatevector.from_counts(result.get_counts(), n_logical_qubits=1, **steane_code)
#     infidelity = 1-(lsv.logical_decomposition[0])**2
#     infidelities_noqec.append(infidelity)

In [ ]:
fig, ax = plt.subplots(dpi=128)

ax.bar(qec_cycle_count_list, infidelities_noqec)

ax.set_title("Infidelity vs. Number of QEC cycles")
ax.set_xlabel("Number of QEC cycles")
ax.set_ylabel(r"$1 - P(0)$")
# ax.set_yscale("log")

plt.show()

### With measurement error correction

In [ ]:
lqc_list = []

for n_qec_cycles in qec_cycle_count_list:
    lqc = lqc_base.copy()
    for _ in range(n_qec_cycles):
        lqc.append_qec_cycle()
    lqc.measure_all(with_error_correction=True)
    lqc_list.append(lqc)

lqc_list[0].draw("mpl")

In [ ]:
shots = 1E5
results = execute_circuits(
  lqc_list,
  backend="aer_simulator",
  hardware_model=hardware_models_Quantinuum["H2-1"], coupling_map=None,
  method="statevector", shots=shots
)

In [ ]:
# Option 1: Logical Counts
logical_counts_list = [lqc.get_logical_counts(result.get_counts()) for lqc, result in zip(lqc_list, results)]
infidelities_qec = [logical_counts.get("1", 0)/shots for logical_counts in logical_counts_list]

# Option 2: LogicalStatevector
# infidelities_qec = []
# for result in results:
#     lsv = LogicalStatevector.from_counts(result.get_counts(), n_logical_qubits=1, **steane_code)
#     infidelity = 1-(lsv.logical_decomposition[0])**2
#     infidelities_qec.append(infidelity)

In [ ]:
fig, ax = plt.subplots(dpi=128)

ax.bar(qec_cycle_count_list, infidelities_qec)

ax.set_title("Infidelity vs. Number of QEC cycles")
ax.set_xlabel("Number of QEC cycles")
ax.set_ylabel(r"$1 - P(0)$")
# ax.set_yscale("log")

plt.show()

### Comparison with and without error-corrected measurements

In [ ]:
# Create the figure and axis objects
fig, ax = plt.subplots(figsize=(14, 7))

# Set the x-axis positions for the bars
x_indices = np.arange(len(qec_cycle_count_list))

# Width of each bar
bar_width = 0.35

# Plot the bars for "With QEC"
bars_with_qec = ax.bar(x_indices - bar_width/2, w_qec_infidelities, bar_width, label="With QEC")

# Plot the bars for "Without QEC" (Orange)
bars_without_qec = ax.bar(x_indices + bar_width/2, wo_qec_infidelities, bar_width, label="Without QEC")

# Format plot
ax.set_title("Comparison of Error Rate With and Without QEC", fontsize=16)

ax.set_xlabel("Number of QEC Cycles", fontsize=12)
ax.set_ylabel("1-P(0)", fontsize=12)

ax.set_xticks(x_indices)
ax.set_xticklabels(qec_cycle_count_list, rotation=45, ha="right")

ax.legend()

ax.yaxis.grid(True, linestyle="--", alpha=0.7)

# Add value labels on top of each bar
def autolabel(bars):
    for bar in bars:
        infidelity = bar.get_height()
        ax.annotate(
            f"{height:.6f}",
            xy=(bar.get_x() + bar.get_width() / 2, infidelity),
            xytext=(0, 3),
            textcoords="offset points",
            ha="center", va="bottom",
            fontsize=10
        )

# Call the autolabel function for both sets of bars
autolabel(bars_with_qec)
autolabel(bars_without_qec)

fig.tight_layout()

plt.show()